In [1]:
#cd C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\NSL-KDD

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Step 1: Load the CSV data
normal_data = pd.read_csv("df_training.csv")
normal_data = normal_data[normal_data['class3'] == 0]
normal_data2=normal_data
adv_data = pd.read_csv("OFF_CWinf-IIoT.csv")
adv_data = adv_data.drop(adv_data.columns[0], axis=1)

# Step 2: Preprocess the data
# Normalize the data and convert it to PyTorch tensors
normal_data = torch.tensor(normal_data.values[:, :-1], dtype=torch.float32)
adv_data = torch.tensor(adv_data.values[:, :-1], dtype=torch.float32)




C:\Users\Juba\.conda\envs\py3.6\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Step 3: Define the Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, latent_dim),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, input_dim),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# Step 4: Train the Autoencoder on normal data
input_dim = normal_data.shape[1]
latent_dim = 78  # Adjust this
autoencoder = Autoencoder(input_dim, latent_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)

epochs = 50 # Adjust this #5BIM;
for epoch in range(epochs):
    # Forward pass
    outputs = autoencoder(normal_data)
    loss = criterion(outputs, normal_data)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Step 5: Extract the latent spaces
normal_latent = autoencoder.encoder(normal_data)
adv_latent = autoencoder.encoder(adv_data)

# Step 6: Create the GAN model
class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.output_dim = output_dim

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
        )

    def forward(self, z):
        generated_data = self.model(z)
        return generated_data

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.input_dim = input_dim

        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        validity = self.model(x)
        return validity



In [4]:
latent_dim = 78 # Adjust this
output_dim = input_dim  # Adjust this
generator = Generator(latent_dim, output_dim)
discriminator = Discriminator(input_dim)
batch_size=32
# Step 7: Train the GAN model
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

adversarial_loss = nn.BCELoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to device
generator = generator.to(device)
discriminator = discriminator.to(device)

num_epochs =5#0 Adjust this #5:FSFM; BIM;

# Training loop
for epoch in range(num_epochs):
    for i in range(0, len(adv_data), batch_size):
        real_data = normal_data[i:i+batch_size].to(device)
        adv_samples = adv_data[i:i+batch_size].to(device)

        # Adversarial ground truths
        valid = torch.ones((real_data.size(0), 1)).to(device)
        fake = torch.zeros((real_data.size(0), 1)).to(device)

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Generate a batch of fake samples from adversarial samples
        fake_data = generator(adv_samples)

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_data), valid[:real_data.size(0)])
        fake_loss = adversarial_loss(discriminator(fake_data.detach()), fake[:fake_data.size(0)])
        discriminator_loss = (real_loss + fake_loss) / 2

        discriminator_loss.backward(retain_graph=True)
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Generate a batch of fake samples from adversarial samples
        fake_data = generator(adv_samples)

        # Measure generator's ability to fool the discriminator
        generator_loss = adversarial_loss(discriminator(fake_data), valid[:fake_data.size(0)])

        generator_loss.backward(retain_graph=True)
        optimizer_G.step()

        if i % 1000 == 0:
            print(
                f"[Epoch {epoch+1}/{num_epochs}] [Batch {i}/{len(adv_data)}] "
                f"Discriminator Loss: {discriminator_loss.item():.4f} "
                f"Generator Loss: {generator_loss.item():.4f}"
            )



[Epoch 1/5] [Batch 0/79884] Discriminator Loss: 0.6895 Generator Loss: 0.6577
[Epoch 1/5] [Batch 4000/79884] Discriminator Loss: 0.5661 Generator Loss: 0.9654
[Epoch 1/5] [Batch 8000/79884] Discriminator Loss: 0.4438 Generator Loss: 1.3241
[Epoch 1/5] [Batch 12000/79884] Discriminator Loss: 0.5615 Generator Loss: 1.2387
[Epoch 1/5] [Batch 16000/79884] Discriminator Loss: 0.4464 Generator Loss: 1.7239
[Epoch 1/5] [Batch 20000/79884] Discriminator Loss: 0.6009 Generator Loss: 1.2060
[Epoch 1/5] [Batch 24000/79884] Discriminator Loss: 0.5262 Generator Loss: 0.9225
[Epoch 1/5] [Batch 28000/79884] Discriminator Loss: 0.3523 Generator Loss: 1.3349
[Epoch 1/5] [Batch 32000/79884] Discriminator Loss: 0.4160 Generator Loss: 1.0867
[Epoch 1/5] [Batch 36000/79884] Discriminator Loss: 0.2571 Generator Loss: 1.7908
[Epoch 1/5] [Batch 40000/79884] Discriminator Loss: 0.6031 Generator Loss: 0.8553
[Epoch 1/5] [Batch 44000/79884] Discriminator Loss: 0.4209 Generator Loss: 1.0466
[Epoch 1/5] [Batch 480

In [5]:
# Step 8: Generate new samples with the GAN
num_samples = len(adv_data)
z = torch.randn((num_samples, latent_dim)).to(device)
generated_samples = generator(z).detach().cpu()
file_Name='CWinf-XAAE-IIoT.csv'
# Step 9: Decode the generated samples using the Autoencoder
decoded_samples = autoencoder.decoder(generated_samples).detach().cpu()

# Step 10: Save the decoded samples as CSV file
output_data = pd.DataFrame(decoded_samples.numpy(), columns=[f"Feature_{i}" for i in range(decoded_samples.shape[1])])

# Set the label column based on the length of adv_data
output_data['class3'] = np.ones(len(output_data))
output_data.columns = normal_data2.columns
output_data.to_csv(file_Name, index=False)

In [6]:
#normal_data2.columns

In [7]:
import pandas as pd
#file_Name='BIM-XAAE-KDD.csv'
# Load the datasets
data_x1 = pd.read_csv('clean_examples.csv')  # Original dataset
data_x2 = pd.read_csv(file_Name)  # Perturbed dataset
# List of columns to keep in x2
columns_to_keep =['Scr_IP','Scr_port', 'Des_IP', 'Des_port','Servicewebsocket', 'Servicessh', 'Servicesmtp', 'Servicesimple_service_discovery', 'Serviceprivate', 'Serviceother', 'Servicenetbios-ns', 'Servicemysql', 'Servicemqtt', 'Servicemodbus', 'Serviceimap', 'Servicehttps', 'Servicehttp', 'Serviceecho', 'Servicedns', 'Servicedhcp', 'Servicecoap', 'Protocoludp', 'Protocoltcp', 'Protocolicmp', 'Conn_state', 'is_syn_only', 'Is_SYN_ACK', 'is_pure_ack', 'is_with_payload', 'FIN or RST', 'OSSEC_alert', 'Login_attempt', 'Succesful_login', 'File_activity', 'Process_activity', 'read_write_physical.process', 'is_privileged']


# Ensure column names are consistent and exist in both dataframes
columns_to_keep = [col for col in columns_to_keep if col in data_x2.columns and col in data_x1.columns]

# Columns to replace in x2 with values from x1 (all columns not in 'columns_to_keep')
columns_to_replace = [col for col in data_x2.columns if col not in columns_to_keep]

# Replace the values
data_x2[columns_to_replace] = data_x1[columns_to_replace]
# Save the modified x2 dataset
data_x2.to_csv(file_Name, index=False)


In [8]:
import pandas as pd

# Load the CSV file
df = pd.read_csv(file_Name)

# Initialize counters for positive and negative rows
positive_rows = 0
negative_rows = 0

# Iterate through each row and check for negative values
for index, row in df.iterrows():
    if (row < 0).any():  # If any value in the row is negative
        negative_rows += 1
    else:  # If there are no negative values in the row
        positive_rows += 1

# Print the total counts
print(f"Total number of positive rows (no negative values): {positive_rows}")
print(f"Total number of negative rows (at least one negative value): {negative_rows}")


Total number of positive rows (no negative values): 79884
Total number of negative rows (at least one negative value): 0
